<a href="https://colab.research.google.com/github/zhouy185/JSON_VIS_Exercise/blob/main/JSON_Vis_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise: Visualization with OSM

In this exercise, we will modify the codes we used in class to show all **parks** in Hamilton, Ontario on a map.

You need to make the necessary changes to the code to make it work for the new context.

## Area ID query

The following function that returns the OVERPASS area id based on city and country names.

* **NO** need to change anything in the function

In [6]:
import requests

def get_osm_area_id(city_name, country_name=None):
    """
    Gets the OpenStreetMap (OSM) area ID for a given city name, optionally including the country,
    using the Nominatim API.

    Args:
        city_name (str): The name of the city.
        country_name (str, optional): The name of the country. Defaults to None.

    Returns:
        int or None: The OSM area ID if found, otherwise None.
    """
    query = city_name
    if country_name:
        query = f"{city_name}, {country_name}"

    url = f"https://nominatim.openstreetmap.org/search?q={query}&format=json&limit=1"
    headers = {'User-Agent': 'Colab Notebook'} # Add a User-Agent header
    response = requests.get(url, headers=headers)
    data = response.json()

    if data:
        # The Nominatim API returns a list of results. We take the first one.
        # The area ID is often the same as the OSM ID for the place.
        # We can add 3600000000 to the OSM ID to get the area ID for use in Overpass API.
        # See: https://wiki.openstreetmap.org/wiki/Overpass_API/Areas
        osm_id = data[0].get('osm_id')
        if osm_id:
            # Assuming the osm_id is for a boundary or administrative area
            # This might need refinement depending on the exact type of place returned
            area_id = osm_id + 3600000000
            return area_id
    return None

# Example usage (optional - you can remove this if you just want the function definition)
# city = "London"
# country = "UK"
# area_id = get_osm_area_id(city, country)
# if area_id:
#     print(f"The OSM area ID for {city}, {country} is: {area_id}")
# else:
#     print(f"Could not find the OSM area ID for {city}, {country}")

# city = "London"
# country = "Canada"
# area_id = get_osm_area_id(city, country)
# if area_id:
#     print(f"The OSM area ID for {city}, {country} is: {area_id}")
# else:
#     print(f"Could not find the OSM area ID for {city}, {country}")

## Using OVERPASS to find parks in the city

Get the area id and then define the OVERPASS query.

In [ ]:
area_id = get_osm_area_id('Hamilton', country_name='Canada')
area_id

In [53]:
query = """
[out:json][timeout:25];
area(id:3600324211)->.a;
 (
    node["amenity"="cafe"](area.a);
    way["amenity"="cafe"](area.a);
    relation["amenity"="cafe"](area.a);
  );
out center tags;
"""

In [54]:
# Import the library
import requests


# The URL
OVERPASS  = "https://overpass-api.de/api/interpreter"

# Sending the data, and assign the response to a variable r

r = requests.post(OVERPASS, data={"data": query}, timeout=60)

r.status_code

200

Get the json data from the response to the query

In [55]:
data = r.json()
places = data.get('elements')

Convert the JSON data to a data frame to preview the content.

In [ ]:
pd.DataFrame(places)

Convert the JSON data to a clean data frame.

In [ ]:
import pandas as pd

# Create an empty list to store the data for the DataFrame
data_list = []

# Iterate through each element in the 'places' list
for place in places:
    # Extract the required information
    place_type = place.get('type')
    place_id = place.get('id')
    tags = place.get('tags')
    name = tags.get('name')
    cuisine = tags.get('cuisine')
    brand = tags.get('brand')

    # Get latitude and longitude based on element type
    if place_type == 'node':
        lat = place.get('lat')
        lon = place.get('lon')
    else:  # For 'way' and 'relation' types, get coordinates from 'center'
        center = place.get('center')
        lat = center.get('lat')
        lon = center.get('lon')

    # Append the extracted data as a dictionary to the list
    data_list.append({
        'type': place_type,
        'id': place_id,
        'name': name,
        'lat': lat,
        'lon': lon,
        'cuisine': cuisine,
        'brand': brand
    })

# Create the DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

# Display the first few rows of the DataFrame
display(df.head())

## Visualization

First install/update plotly

In [ ]:
!pip install -U plotly

Import plotly

In [20]:
import plotly.express as px

Create the scatter map plot.

In [ ]:
fig = px.scatter_map(
    df, lat="lat", lon="lon",
    color="type",          # or category if available
    hover_name="name",
    hover_data=["brand", "cuisine"],
    map_style="carto-positron",
    zoom=11, height=500
)
fig.show()